In [ ]:
from pymongo import MongoClient

import requests
import json
import pandas as pd

In [ ]:
match_data = fetch_match_details_aoe2recs(12, query)

db_conn = setup_mongo_conn()

insert_match_mongo(match_data,db_conn)